In [11]:
# # Building Name & State Dimensions & Control Dimensions & Num. of Constraints & Main Objective (Minimize) \\
# from energym.wrappers.rl_wrapper import StableBaselinesRLWrapper
# from buildings_factory import *
# objectives = ["average exchanged energy",
#               "average exchanged energy",
#               "average exchanged energy",
#               "average exchanged energy",
#               "power demand",
#               "power demand",
#               "CO2 emission",
#               "CO2 emission",
#               "power demand",
#               "power demand",
#               "power demand",
#               "power demand"]

# model_info = ""
# for building_name in buildings_list:
#     # min_kpis, max_kpis = collect_baseline_kpi(building_name)
#     building_idx = buildings_list.index(building_name)
#     env = get_env(building_name)
#     inputs = get_inputs(building_name, env)
#     default_control = default_controls[building_idx]
#     env_rl = StableBaselinesRLWrapper(building_name, None, None, reward_func)
#     input_dim = env_rl.observation_space.shape[0]
#     control_dim = env_rl.action_space.shape[0]
#     constraint_num = 0
#     for key, val in env.kpi_options.items():
#         if val["type"] == 'avg_dev': constraint_num += 1 
#     main_objective = objectives[building_idx]
#     model_info += f"{building_name} & {input_dim} & {control_dim} & {constraint_num} & {main_objective} \\ "
# print(model_info)

In [12]:
import pandas as pd
# buildings_list = ["ApartmentsThermal-v0", "ApartmentsGrid-v0", "Apartments2Thermal-v0",
#                   "Apartments2Grid-v0", "OfficesThermostat-v0", "MixedUseFanFCU-v0",
#                   "SeminarcenterThermostat-v0", "SeminarcenterFull-v0", "SimpleHouseRad-v0",
#                   "SimpleHouseRSla-v0", "SwissHouseRSlaW2W-v0", "SwissHouseRSlaTank-v0"] 

# plot_building_names = ["SimpleHouseRad-v0", "SimpleHouseRSla-v0", 
#                         "SwissHouseRSlaW2W-v0", "SwissHouseRSlaTank-v0"]

plot_building_names = ["OfficesThermostat-v0", "MixedUseFanFCU-v0"]
seeds = [7, 13, 17, 19]

# "MOPO_ori_simulator"
# model_names = ["TD3PlusBC_ori_simulator", "CQL_ori_simulator", "MOPO_ori_simulator",
#                "SAC_bs_predictor", "SAC_4_dnn_predictor", "SAC_8_dnn_predictor"]
# legend_names = ["TD3PlusBC", "CQL", "MOPO", "Best", "Ours_4", "Ours_8"]

len_traj = 1

# model_names = ["SAC_{}_dnn_predictor{}_alpha1", "SAC_{}_dnn_predictor{}_alpha2",
#                "SAC_{}_dnn_predictor{}_alpha4", "SAC_{}_dnn_predictor{}_alpha6",
#                "SAC_{}_dnn_predictor{}_alpha8", "SAC_{}_dnn_predictor{}_alpha10"]
# legend_names = ["alpha1" , "alpha2", "alpha4", "alpha6", "alpha8", "alpha10"] 

model_names = ["SAC_{}_dnn_predictor_alpha1{}", "SAC_{}_dnn_predictor_alpha2{}",
               "SAC_{}_dnn_predictor_alpha4{}", "SAC_{}_dnn_predictor_alpha6{}",
               "SAC_{}_dnn_predictor_alpha8{}", "SAC_{}_dnn_predictor_alpha10{}"]
legend_names = ["alpha1" , "alpha2", "alpha4", "alpha6", "alpha8", "alpha10"] 

# model_names = ["TD3PlusBC_ori_simulator", "CQL_ori_simulator", "SAC_bs_predictor", "SAC_dnn_predictor"]
# legend_names = ["TD3PlusBC", "CQL", "Best", "Ours"]
res_cols = ["manual_baseline_eval_episode_reward", "manual_eval_episode_reward"]
            
colors = ['r', 'g', 'r--', 'g--']

res_data = []
for seed in seeds:
    for _model_name, legend_name in zip(model_names, legend_names):
        for building in plot_building_names:
            model_name = _model_name.format(len_traj, "_seed%i"%seed)
            res_loc = f"data/data_aml/models/{building}/{model_name}/logs/best/rewards.csv"
            res_row = pd.read_csv(res_loc).iloc[-1, :]
            model_col_name = _model_name.format(len_traj, "")
            res_data.append([seed, building, model_col_name, legend_name,
                             res_row["eval_episode_reward"], res_row["manual_eval_episode_reward"],
                             res_row["baseline_eval_episode_reward"], res_row["manual_baseline_eval_episode_reward"]])
res_df = pd.DataFrame(res_data, columns=["seed", "building", "model_name", 
                                         "legend", "learnt_reward", "manual_reward",
                                         "learnt_baseline_reward", "manual_baseline_reward"])
            

res_df.head()

,seed,building,model_name,legend,learnt_reward,manual_reward,learnt_baseline_reward,manual_baseline_reward
0,7,OfficesThermostat-v0,SAC_1_dnn_predictor_alpha1,alpha1,17375.524580,-9158.263471,9878.566254,-24963.065866
1,7,MixedUseFanFCU-v0,SAC_1_dnn_predictor_alpha1,alpha1,43368.054260,-4839.341928,17193.870300,-9292.691599
2,7,OfficesThermostat-v0,SAC_1_dnn_predictor_alpha2,alpha2,17359.839235,-13101.102083,7704.398046,-24958.793086
3,7,MixedUseFanFCU-v0,SAC_1_dnn_predictor_alpha2,alpha2,44110.899474,-4191.831879,15137.535276,-9292.741633
4,7,OfficesThermostat-v0,SAC_1_dnn_predictor_alpha4,alpha4,11927.841646,-29529.403081,3329.096220,-24978.089031


In [14]:
import numpy as np
cols = res_df.columns.tolist()
# "RL_reward", "manual_RL_reward", "baseline_reward", "manual_baseline_reward"
cols.remove("seed")
stat_df = res_df[cols].groupby(["building", "model_name", "legend"]).agg(learnt_reward_mean=("learnt_reward", np.mean),
                                                           learnt_reward_std=("learnt_reward", np.std),
                                                           learnt_baseline_reward_mean=("learnt_baseline_reward", np.mean),
                                                           learnt_baseline_reward_std=("learnt_baseline_reward", np.std),
                                                           manual_reward_mean=("manual_reward", np.mean),
                                                           manual_reward_std=("manual_reward", np.std),
                                                           manual_baseline_reward_mean=("manual_baseline_reward", np.mean),
                                                           manual_baseline_reward_std=("manual_baseline_reward", np.std)).reset_index()

stat_df.fillna(0.0, inplace=True)
stat_df

,building,model_name,legend,learnt_reward_mean,learnt_reward_std,learnt_baseline_reward_mean,learnt_baseline_reward_std,manual_reward_mean,manual_reward_std,manual_baseline_reward_mean,manual_baseline_reward_std
0,MixedUseFanFCU-v0,SAC_1_dnn_predictor_alpha1,alpha1,44791.151781,1110.778863,17204.371672,15.466174,-7114.165774,1847.276083,-9297.561040,9.072327
1,MixedUseFanFCU-v0,SAC_1_dnn_predictor_alpha10,alpha10,26312.987025,3873.330561,-1383.365449,2.904743,-6666.030279,2641.820945,-9299.712890,1.447231
2,MixedUseFanFCU-v0,SAC_1_dnn_predictor_alpha2,alpha2,43773.132471,1250.695062,15144.147511,7.793472,-4899.122614,859.934143,-9303.113575,16.522422
3,MixedUseFanFCU-v0,SAC_1_dnn_predictor_alpha4,alpha4,37826.575462,3597.102714,11027.450784,23.240378,-7035.442189,2080.960397,-9296.436109,15.755487
4,MixedUseFanFCU-v0,SAC_1_dnn_predictor_alpha6,alpha6,37816.509199,1786.279453,6896.986077,28.455362,-7748.791155,2022.611033,-9301.217997,10.632095
5,MixedUseFanFCU-v0,SAC_1_dnn_predictor_alpha8,alpha8,27527.046349,8297.085904,2768.351582,22.690913,-6398.594310,1539.405653,-9300.776537,8.639141
6,OfficesThermostat-v0,SAC_1_dnn_predictor_alpha1,alpha1,19115.867319,3666.322617,9891.889899,14.129928,-29453.676650,28314.447999,-24985.045136,17.684000
7,OfficesThermostat-v0,SAC_1_dnn_predictor_alpha10,alpha10,580.979680,2058.023964,-9829.225834,28.944299,-20243.161668,8542.420235,-24979.032839,14.780976
8,OfficesThermostat-v0,SAC_1_dnn_predictor_alpha2,alpha2,16226.452160,869.039172,7699.096842,8.891690,-17684.319801,7142.165251,-24981.620855,19.470139
9,OfficesThermostat-v0,SAC_1_dnn_predictor_alpha4,alpha4,12379.109139,1629.215100,3334.982169,11.106545,-22050.061669,5802.432522,-24981.603394,19.342431


In [15]:
bs_stat_df = stat_df[['building', "manual_baseline_reward_mean"]].groupby("building").mean().reset_index()
bs_stat_df.columns = ['building', 'manual_reward_mean']
bs_stat_df['legend'] = 'baseline'
bs_stat_df['manual_reward_std'] = 0.0
bs_stat_df.head()

,building,manual_reward_mean,legend,manual_reward_std
0,MixedUseFanFCU-v0,-9299.803025,baseline,0.0
1,OfficesThermostat-v0,-24981.550769,baseline,0.0


In [16]:
cols = ["building", "legend", "manual_reward_mean", "manual_reward_std"]
stat_df_all = pd.concat([stat_df[cols], bs_stat_df[cols]])
stat_df_all['reward'] = stat_df_all.apply(lambda x: (f"{x['manual_reward_mean']:.0f}\xb1{x['manual_reward_std']:.0f}" if x['legend'] != "baseline" else f"{x['manual_reward_mean']:.0f}"), axis=1)
df = stat_df_all.pivot(index='building', columns='legend', values='reward').reset_index()


In [17]:
print_cols = ["building", "baseline", "alpha1", "alpha2", "alpha4", "alpha6", "alpha8", "alpha10"]
# print_cols = ["building", "baseline", "CQL", "TD3PlusBC", "MOPO", "Ours_8"]
print(" & ".join(print_cols), "\\\\")
print("\\hline")
for _, row in df.iterrows():
    print(" & ".join([str(row[col]) for col in print_cols]), "\\\\")
    print("\\hline")

building & baseline & alpha1 & alpha2 & alpha4 & alpha6 & alpha8 & alpha10 \\
\hline
MixedUseFanFCU-v0 & -9300 & -7114±1847 & -4899±860 & -7035±2081 & -7749±2023 & -6399±1539 & -6666±2642 \\
\hline
OfficesThermostat-v0 & -24982 & -29454±28314 & -17684±7142 & -22050±5802 & -24540±17477 & -17001±8013 & -20243±8542 \\
\hline
